In [1]:
import skmob
from skmob.utils import utils, constants
import pandas as pd
import geopandas as gpd
import numpy as np
from skmob.models import Gravity
import Levenshtein as Lv

##############################################################
#Paths
##############################################################

# INPUTS

# File downloaded from https://github.com/openpolis/geojson-italy/blob/master/geojson/limits_IT_provinces.geojson,
# The file contains information about the locations and ids of Italian provinces
in_Geojson_Provinces_IT = "input\\Italy_Provinces_Geojson.json"



# File downloaded from https://data.humdata.org/dataset/covid-19-mobility-italy. The populations for each province were 
# edited from https://www.citypopulation.de/en/italy/admin/
in_Italian_ProvincesID_Population_Path = "input\\Italy_ProvincesID_Population.csv"

# File downloaded from https://data.humdata.org/dataset/covid-19-mobility-italy. Contains origin-destination provinces
# from Italy, on a time scale
in_Origin_Destination_Matrix_Path = "input\\Italy_origin_destination_matrix.csv"

##############################################################

# OUTPUTS 

# The convered file from the geojson. Contains only the province id and the geometry of the province
out_ProvinceID_Geometry_CSV_Path = "output\\Italy_ProvinceID_Geometry.csv"

# Merged file which contains the Italian provinces along with population and ids
out_ProvinceID_Geometry_Population_Path = "output\\Italy_ProvinceID_Geometry_Population.csv"

# Path of the origin-destination Matrix created by Python
out_Origin_Dest_Flows_Path = "output\\Italy_Origin_Destination_Flows_Date.csv"
out_Origin_Dest_Flows_Mini_Path = "output\\Italy_Origin_Destination_Flows_Date_Mini.csv"
#############################################################
# Read geojson file containing provinces locations
geoJson_Path = open(in_Geojson_Provinces_IT)
gpd_df_geojson = gpd.read_file(geoJson_Path)

# Delete extra columns from geojson dataframe
del gpd_df_geojson['prov_istat_code_num']
del gpd_df_geojson['prov_acr']
del gpd_df_geojson['reg_istat_code_num']
del gpd_df_geojson['reg_istat_code']
del gpd_df_geojson['prov_name']
del gpd_df_geojson['reg_name']



# Read the provinces with population file
it_dtset_file = open(in_Italian_ProvincesID_Population_Path)
pd_df_ProvID_Population = pd.read_csv(it_dtset_file)

# Convert the geopandas to pandas dataframe, in order to merge it with the population dataframe
pd_Geojson = pd.DataFrame(gpd_df_geojson)

# Merge dataframes based on province id
pd_Geojson['prov_istat_code']=pd_Geojson['prov_istat_code'].astype(int)
df_Geometry_ProvID_Population = pd.merge(pd_Geojson,pd_df_ProvID_Population,on='prov_istat_code')
df_Geometry_ProvID_Population.columns = ['tile_ID', 'geometry', 'index', 'province_code', 'province_name', 'population']
df_Geometry_ProvID_Population.pop('index')
df_Geometry_ProvID_Population.pop('province_code')
tessellation = gpd.GeoDataFrame(df_Geometry_ProvID_Population, geometry=df_Geometry_ProvID_Population.geometry, crs = 'epsg:4326')

In [2]:
out_Origin_Dest_Flows_Path_January_1 = "output\\aggregated_flows\\2020_01_20_2020_01_26_aggregated.csv"
out_Origin_Dest_Flows_Path_January_2 = "output\\aggregated_flows\\2020_01_27_2020_02_02_aggregated.csv"
out_Origin_Dest_Flows_Path_March_1 = "output\\aggregated_flows\\2020_03_30_2020_04_05_aggregated.csv"
out_Origin_Dest_Flows_Path_March_2 = "output\\aggregated_flows\\2020_04_06_2020_04_12_aggregated.csv"
out_Origin_Dest_Flows_Path_March_2 = "output\\aggregated_flows\\2020_03_30_2020_04_05_aggregated.csv"
df1 = pd.read_csv(out_Origin_Dest_Flows_Path_January_1)
df2 = pd.read_csv(out_Origin_Dest_Flows_Path_January_2)


# LOD,NLOD = Lv.calculateLOD(df1,df2)
# mNLOD = Lv.calculateMean(NLOD)
# print(mNLOD)

# df1 = pd.read_csv(out_Origin_Dest_Flows_Path_March_1)
# df2 = pd.read_csv(out_Origin_Dest_Flows_Path_March_2)


# LOD,NLOD = Lv.calculateLOD(df1,df2)
# mNLOD = Lv.calculateMean(NLOD)
# print(mNLOD)

In [3]:
fdf = skmob.FlowDataFrame.from_file(out_Origin_Dest_Flows_Path_March_2,
                                    tessellation=tessellation,
				                    tile_id='tile_ID',
                                    datetime = 'date',
                                    origin = 'origin',
                                    destination = 'destination',
                                    flow = 'flow',
				                    sep=",")
tot_outflows = fdf[fdf['origin'] != fdf['destination']].groupby(by='origin', axis=0)['flow'].sum().fillna(0).values
tessellation[constants.TOT_OUTFLOW] = tot_outflows
print(tessellation.head())

   tile_ID                                           geometry province_name  \
0        1  POLYGON ((7.89397 45.58222, 7.89654 45.57985, ...        Torino   
1        2  POLYGON ((7.92900 45.74244, 7.92584 45.74196, ...      Vercelli   
2        3  POLYGON ((8.42079 45.82981, 8.42028 45.83010, ...        Novara   
3        4  MULTIPOLYGON (((6.94540 44.42794, 6.94734 44.4...         Cuneo   
4        5  POLYGON ((7.96685 45.11667, 7.96729 45.11673, ...          Asti   

   population  tot_outflow  
0     2259523       276359  
1      170911       101058  
2      369018       190796  
3      587089        40300  
4      214638        27360  


In [7]:
out_Origin_Dest_Flows_Path_January_1 = "output\\aggregated_flows\\2020_01_20_2020_01_26_aggregated.csv"
out_Origin_Dest_Flows_Path_January_2 = "output\\aggregated_flows\\2020_01_27_2020_02_02_aggregated.csv"
out_Origin_Dest_Flows_Path_January_3 = "output\\aggregated_flows\\2020_02_03_2020_02_09_aggregated.csv"
out_Origin_Dest_Flows_Path_March_1 = "output\\aggregated_flows\\2020_03_23_2020_03_29_aggregated.csv"
out_Origin_Dest_Flows_Path_March_2 = "output\\aggregated_flows\\2020_03_30_2020_04_05_aggregated.csv"
df1 = pd.read_csv(out_Origin_Dest_Flows_Path_January_1)
df2 = pd.read_csv(out_Origin_Dest_Flows_Path_January_2)
df3 = pd.read_csv(out_Origin_Dest_Flows_Path_January_3)

df1 = pd.read_csv(out_Origin_Dest_Flows_Path_March_2)

deterr_arg = np.arange(-3.5, -3, 0.05).tolist()
deterr_arg = [ round(elem, 2) for elem in deterr_arg ]
print(len(deterr_arg))
dest_exp = np.arange(0.85,1.01,.05).tolist()
dest_exp = [ round(elem, 2) for elem in dest_exp ]
txt = "deterrence_args,destination_exp,mNLOD\n"
for deterrence in deterr_arg:
    for destination in dest_exp:
        gravity_singly = Gravity(gravity_type='singly constrained',
                                deterrence_func_args=[deterrence], 
                                origin_exp=1, 
                                destination_exp=destination,
                                deterrence_func_type='power_law')

        np.random.seed(0)

        synth_fdf = gravity_singly.generate(tessellation,
                                        tile_id_column='tile_ID',
                                        tot_outflows_column='tot_outflow',
                                        relevance_column= 'population',
                                        out_format='flows')
        synth_fdf['flow'] = synth_fdf['flow'].astype(int)

        df_path = "hello_march.csv"
        synth_fdf.to_csv(df_path, index = False)
        df4 = pd.read_csv(df_path)

        LOD,NLOD = Lv.calculateLOD_Common_v2(df1,df4)
        mNLOD = Lv.calculateMean(NLOD)
        txt += str(deterrence)+","+str(destination)+","+str(mNLOD)+"\n"
        print(deterrence,destination,mNLOD)
print()
print(txt)

100%|██████████| 107/107 [00:00<00:00, 2548.66it/s]10

C:\Users\kostk\.conda\envs\skmob\lib\site-packages\skmob\models\gravity.py:43: RuntimeWarning: divide by zero encountered in power
  return np.power(x, exponent)
100%|██████████| 107/107 [00:00<00:00, 3243.57it/s]-3.5 0.85 0.527613727185158

C:\Users\kostk\.conda\envs\skmob\lib\site-packages\skmob\models\gravity.py:43: RuntimeWarning: divide by zero encountered in power
  return np.power(x, exponent)
100%|██████████| 107/107 [00:00<00:00, 2675.86it/s]-3.5 0.9 0.5264729376515177

C:\Users\kostk\.conda\envs\skmob\lib\site-packages\skmob\models\gravity.py:43: RuntimeWarning: divide by zero encountered in power
  return np.power(x, exponent)
100%|██████████| 107/107 [00:00<00:00, 3822.52it/s]-3.5 0.95 0.5282024533887535

C:\Users\kostk\.conda\envs\skmob\lib\site-packages\skmob\models\gravity.py:43: RuntimeWarning: divide by zero encountered in power
  return np.power(x, exponent)
100%|██████████| 107/107 [00:00<00:00, 3243.36it/s]-3.5 

In [5]:
print(tessellation.head(100))

    tile_ID                                           geometry  province_name  \
0         1  POLYGON ((7.89397 45.58222, 7.89654 45.57985, ...         Torino   
1         2  POLYGON ((7.92900 45.74244, 7.92584 45.74196, ...       Vercelli   
2         3  POLYGON ((8.42079 45.82981, 8.42028 45.83010, ...         Novara   
3         4  MULTIPOLYGON (((6.94540 44.42794, 6.94734 44.4...          Cuneo   
4         5  POLYGON ((7.96685 45.11667, 7.96729 45.11673, ...           Asti   
..      ...                                                ...            ...   
95       83  MULTIPOLYGON (((15.44299 38.02576, 15.44250 38...        Messina   
96       84  MULTIPOLYGON (((13.66344 37.19338, 13.66190 37...      Agrigento   
97       85  MULTIPOLYGON (((13.67698 37.56759, 13.67126 37...  Caltanissetta   
98       86  MULTIPOLYGON (((14.67399 37.55798, 14.66984 37...           Enna   
99       87  MULTIPOLYGON (((15.16974 37.57438, 15.16971 37...        Catania   

    population  tot_outflow

In [6]:
gravity_singly_fitted = Gravity(gravity_type='singly constrained')
print()
print(gravity_singly_fitted)

gravity_singly_fitted.fit(fdf, relevance_column='population')
print(gravity_singly_fitted)

np.random.seed(0)
synth_fdf_fitted = gravity_singly_fitted.generate(tessellation,
                                                        tile_id_column='tile_ID',
                                                        tot_outflows_column='tot_outflow',
                                                        relevance_column= 'population',
                                                        out_format='flows')

print(synth_fdf_fitted.head())


Gravity(name="Gravity model", deterrence_func_type="power_law", deterrence_func_args=[-2.0], origin_exp=1.0, destination_exp=1.0, gravity_type="singly constrained")
100%|██████████| 107/107 [00:00<00:00, 2816.68it/s]Gravity(name="Gravity model", deterrence_func_type="power_law", deterrence_func_args=[-2.558684917731057], origin_exp=1.0, destination_exp=0.7798422620969063, gravity_type="singly constrained")
  origin destination           flow
0      1           2   65240.074458
1      1           3   55985.247723
2      1           4  161680.538937
3      1           5  103728.428847
4      1           6   56470.866867

C:\Users\kostk\.conda\envs\skmob\lib\site-packages\skmob\models\gravity.py:43: RuntimeWarning: divide by zero encountered in power
  return np.power(x, exponent)


In [6]:
deterr_arg = np.arange(-3, -1.95, 0.05).tolist()
deterr_arg = [ round(elem, 2) for elem in deterr_arg ]
print(deterr_arg)
print(len(deterr_arg))

dest_exp = np.arange(0.5,1.01,.05).tolist()
dest_exp = [ round(elem, 2) for elem in dest_exp ]
print(dest_exp)
print(len(dest_exp))

[-3.0, -2.95, -2.9, -2.85, -2.8, -2.75, -2.7, -2.65, -2.6, -2.55, -2.5, -2.45, -2.4, -2.35, -2.3, -2.25, -2.2, -2.15, -2.1, -2.05, -2.0]
21
[0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0]
11


In [5]:
file_path = "mNLOD_comparison\\Mar1_b.txt"
Origin_Destination_File = open(file_path, "w")
Origin_Destination_File.write(txt)
Origin_Destination_File.close()

In [1]:
df1 = pd.read_csv("mNLOD_comparison\\Mar1_b.txt")
df2 = pd.read_csv("mNLOD_comparison\\Mar2_b.txt")
print(df1.head())
print(df2.head())

df1 = df1.append(df2, ignore_index = True)
df1 = df1.sort_values(['deterrence_args','destination_exp'])
df1.to_csv("mNLOD_comparison\\March_Merged_b.csv", index = False)

df1 = df1.sort_values(['mNLOD'])
df1.to_csv("mNLOD_comparison\\March_Merged_Sort_mNLOD_b.csv", index = False)
print(df1)

NameError: name 'pd' is not defined